# Оценка качества синтеза модели Parler-TTS Mini v0.1 - Jenny

## Шаг 1. Метрики
Оценивать синтез речи, произведённый моделью, будем по следующим характеристикам: схожесть со спикером (speaker-similarity), качество речи (speech quality), надежность (robustness, то есть оценка того, насколько транскрипция сгенерированной аудизаписи содержательно схожа с референсом), скорость инференса. Для оценки модели по описанным выше характеристикам были выбраны следующие метрики:

* speaker-similarity: SIM-O. Эта метрика была выбрана, как более объективная и воспроизводимая по сравнению с аналоговой SIM-R, более подробно можно почитать в разделе SIM [этой исследовательской работы](https://arxiv.org/pdf/2407.08551).
* speech quality: MOS, запредикченная системой [UTMOS](https://github.com/sarulab-speech/UTMOS22), выбранной по [результатам](https://arxiv.org/pdf/2203.11389) сравнения с аналогами по объективным показателям
* robustness: WER (Word Error Rate), выбранная как метрика объективная, легкая для замера и интерпретации.

## Шаг 2. Работа с данными
Для выбора оптимальных для оценки синтеза модели выделенными метриками аудиозаписей из датасета были обозначены следующие принципы:

* Синтаксическое разнообразие: тексты разной структуры и длины
* Лексическое разнообразие: различные стили речи и темы
* Фонетическое разнообразие: содержание широго спектра звуков и фонем

Загрузим датасет, возьмём рандомизированный семпл тысячи записей для оптимизации дальнейшей выборки.

In [ ]:
from datasets import load_dataset
import random

ds = load_dataset("reach-vb/jenny_tts_dataset")
batch_size = 1000
sampled_data = ds['train'].shuffle(seed=42).select(range(1000))
del ds

Как работает оценка аудиозаписей по выбранным параметрам?<br>
`cmudict` из модуля `nltk` для процессинга натурального языка позволяет извлечь фонемы из слова, обернём это в вспомогательную функцию `get_unique_phonetics`.<br>
Основная функция `select_diverse_sentences` предназначена для извлечения `num_to_select` записей из датасета таких, что разброс следующих трех параметров среди них максимален: процент пересечения слов с уже выбранными, процент пересечения фонем с уже выбранными, длина предложения. Для этого был написан жадный алгоритм, отбирающий на каждом шаге лучший вариант аудиозаписи (т.е. самый отдаленный) из всех оставшихся.

In [ ]:
import string
import nltk


nltk.download('cmudict')
from nltk.corpus import cmudict
phonetics = cmudict.dict()


def preprocess_sentence(sentence): # Пре-процессинг предложений: убираем пунктуацию
    return sentence.translate(str.maketrans('', '', string.punctuation))
def get_unique_phonetics(words):
    unique_phonetics = set()
    for word in words:
        phonetic = phonetics.get(word.lower()) # Сохраняем все фонемы каждого из слов в сет фонем всего предложения
        if phonetic: # Некоторых слов может не быть в словаре
            for ph in phonetic[0]:
                unique_phonetics.add(ph)
    return unique_phonetics # Возвращаем полученные уникальные фонемы


    
def select_diverse_sentences(sequence, num_to_select): # Выбираем num_to_select максимально удаленных друг от друга записей по параметрам длины, лексики и звуков

    selected_audios = [] # Список выбранных записей

    sounds = []
    words = []
    
    for entry in sequence: # Предварительно итерируемся по всему датасету, чтобы для каждой записи вычислить сеты её фонем и слов
        sentence = set(preprocess_sentence(entry['transcription_normalised']).split())
        sounds.append(get_unique_phonetics(sentence))
        words.append(sentence)

    filenames = set() # Для проверки того, взяли ли мы какую-то запись (имена файлов в датасете уникальны)
    taken_sounds = set()
    taken_words = set()
    taken_length_sum = 0
    taken_amount = 0

    def update(index): # Функция добавления свеже-выбранной записи
        nonlocal selected_audios, filenames, taken_sounds, taken_words, taken_length_sum, taken_amount
        selected_audios.append(sequence[index])
        filenames.add(sequence[index]['file_name'])
        taken_sounds |= sounds[index]
        taken_words |= words[index]
        taken_length_sum += len(words[index])
        taken_amount += 1

    update(random.randint(0, len(sequence)-1)) # Первая запись выбрана случайно
    for time in range(num_to_select-1):
        print(f"{len(selected_audios)}/{num_to_select} аудиозаписей выбрано.")
        max_distance = float('-inf') # Критерий выбора (сумма значений всех параметров)
    
        for index in range(len(sequence)):
            if sequence[index]['file_name'] not in filenames:
                # Отношение пересечения сетов слов предложения и всех взятых к сету слов этого предложения
                # (пытаемся выбрать такое предложение, которое привносит максимальную долю еще не встреченных нами слов)
                lexical_coeff = len(words[index] - taken_words)/len(words[index])*1000 

                
                if sounds[index]:
                    # Похожая схема, но с долей фонем
                    phonetical_coeff = len(sounds[index] - taken_sounds)/len(sounds[index])*1000
                else:
                    phonetical_coeff = 0

                #Пытаемся выбрать такое предложение, длина которого внесет наибольших вклад в среднее арифметическое длин
                curr_mean = taken_length_sum/taken_amount
                try:
                    length_coeff = (curr_mean/abs(curr_mean - (taken_length_sum+len(words[index]))/(taken_amount+1)))*1000
                except ZeroDivisionError:
                    length_coeff = 0

                    
                distance = lexical_coeff + phonetical_coeff + length_coeff

                # Нашли вариант лучше выбранного в данный момент
                if distance > max_distance:
                    max_distance = distance
                    best_index = index

        update(best_index)

    return selected_audios



Выберем 100 оптимальных для нас записей.

In [ ]:
dataset = select_diverse_sentences(sampled_data, 100)
del sampled_data # Удаляем неиспользуемые переменные, чтобы освободить память
del phonetics

Получили датасет, на котором будем оценивать модель.
## Шаг 3. Инференс
Сгенерируем аудиозаписи из полученных нами на предыдущем шаге промптов из датасета.
На этом шаге нам было бы важно поэксперементировать с текстовым описанием, передаваемым в модель, если бы датасет, на котором тренировали модель был бы разнообразным: разные спикеры, эмоции, бэкграунд-звуки; но это не наш случай. Было подобрано нейтральное описание, соответствующее общему тону аудиозаписей датасета.

In [ ]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf
import time

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-jenny-30H").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-jenny-30H")

inferences = []
description = "Jenny speaks in a very confined sounding environment with clear audio quality."
input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
for index, entry in enumerate(dataset):
    start_time = time.time() # Для замера времени инференса
    
    prompt = entry['transcription_normalised']
    prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids) # Генерируем синтезированную речь 
    
    audio_arr = generation.cpu().numpy().squeeze()
    entry['audio']['array_synthesized'] = audio_arr # Сохраняем векторное представление записи
    inference_time = time.time() - start_time
    audio_duration = len(audio_arr) / model.config.sampling_rate  # Вычисляем длительность записи в секундах
    inferences.append(inference_time / audio_duration)
    sf.write(f"{index}_synthesized.wav", audio_arr, model.config.sampling_rate) # Сохраняем аудиозапись в .wav
    print(f"{index+1}/{len(dataset)} записей обработано.")

## Шаг 4. Проводим оценку модели
Самая интересная часть проекта, в которой нам необходимо произвести техническую реализацию оценки модели по выбранным нам характеристикам: натуральность звучания речи, схожесть звучания со спикером, содержательное совпадение промпта с транскрипцией речи модели.<br>
### MOS - Mean Opinion Score
Эта метрика представляет из себя человеческую субъективную оценку натуральности синтезированной речи, что значит, что автоматизировать её алгоритмически - очень сложная, если не невозможная задача. Очевидно, для этого нам нужно использовать пре-тренированные модели. Лучшим вариантом, как уже было сказано выше, оказалась система UTMOS. Примечательно, что несмотря на то, что её код целиком написан на Python, у системы UTMOS нет Python API из коробки - только CLI. Поэтому рабочим вариантом стал репозиторий [SpeechMOS](https://github.com/tarepan/SpeechMOS).
### WER - Word Error Rate
Крайне простая метрика в замере метрика - при подаче нам двух текстов: оригинала и синтезированного, нам нужно учесть, какая доля слов была "проглочена" в процессе синтеза, какая - придумана, и, наконец, какая изменена.<br>
Проблема заключается лишь в том, чтобы эти текста получить: оригинал нам уже дан, а вот синтезированный нам придется, очевидно, доставать из сгенерированной нами на предыдущем шаге аудиозаписи, т.е. произвести транскрибацию. В этом нам поможет модель `hubert-large-ls960-ft`, с помощью которой мы сначала пре-процессим векторное представление записи для соответствия формату входных данных модели `CTC`, а потом скармливаем полученный массив, собственно, `hubert`. 
### SIM - SIMilarity with the Original
Для этой метрики нам понадобится прогонять наши представления аудиозаписей через некоторую модель, чтобы получить эмбеддинг этой самой записи. С этим хорошо справляется `wav2vec-large-960h`, загруженная через дефолтную base-модель модуля transformers. Затем - просто высчитать расстояние между двумя векторами.

In [ ]:
import os

import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2Model, HubertForCTC
from scipy.spatial.distance import cosine # Для замера расстояния между векторами

mos_predictor = torch.hub.load("tarepan/SpeechMOS:v1.2.0", "utmos22_strong", trust_repo=True) # MOS predictor from UTMOS
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft") # For transcription (WER)
embedding_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h") # For embedding (SIM-O)
transcription_model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft") # For transcription (WER)

def get_embeddings(waveform, sample_rate):
    waveform = torch.from_numpy(waveform).float()
    
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
    
    inputs = waveform.unsqueeze(0)
    with torch.no_grad():
        outputs = embedding_model(inputs).last_hidden_state.mean(dim=1).squeeze() # Прогоняем через модель
    
    # Возвращаем эмбеддинг
    return outputs

def get_transcription(waveform, sample_rate):
    waveform = torch.from_numpy(waveform).float()

    if sample_rate != 16000: # hubert принимает лишь записи с sample_rate 16000, так что наши придется переконвертировать
        waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
    
    input_values = processor(waveform, return_tensors="pt", sampling_rate=16000).input_values # Пре-процессим
    logits = transcription_model(input_values).logits # Передаем в модель
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]) # Декодируем в человекочитаемый язык
    return transcription

def preprocess(text): # Для нормализации транскрибаций
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def wer(reference, hypothesis): # Алгоритм высчитывания Word Error Rate
    reference = preprocess(reference)
    hypothesis = preprocess(hypothesis)
    ref_words = reference.split()
    hyp_words = hypothesis.split()

    d = [[0] * (len(hyp_words) + 1) for _ in range(len(ref_words) + 1)]

    for i in range(len(ref_words) + 1):
        d[i][0] = i  # Deletion cost
    for j in range(len(hyp_words) + 1):
        d[0][j] = j  # Insertion cost

    for i in range(1, len(ref_words) + 1):
        for j in range(1, len(hyp_words) + 1):
            cost = 0 if ref_words[i - 1] == hyp_words[j - 1] else 1
            d[i][j] = min(d[i - 1][j] + 1,      # Deletion
                           d[i][j - 1] + 1,      # Insertion
                           d[i - 1][j - 1] + cost)  # Substitution

    S = d[len(ref_words)][len(hyp_words)]
    N = len(ref_words)
    wer_value = S / N if N > 0 else 1

    return wer_value

In [ ]:
metrics = {} # Словарь для удобного хранения метрик каждой записи

for index, entry in enumerate(dataset):

    # Высчитываем SIM
    original_embedding = get_embeddings(entry['audio']['array'], model.config.sampling_rate)
    generated_embedding = get_embeddings(entry['audio']['array_synthesized'], model.config.sampling_rate)
    similarity = 1 - cosine(original_embedding.numpy(), generated_embedding.numpy())

    # Высчитываем WER
    original_transcription = entry['transcription_normalised']
    print("Original transcription:",original_transcription)
    generated_transcription = get_transcription(entry['audio']['array_synthesized'], model.config.sampling_rate)
    print("Generated transcription:",generated_transcription)
    wer_value = wer(original_transcription, generated_transcription)

    # "Высчитываем" MOS :)
    mos = mos_predictor(torch.from_numpy(entry['audio']['array_synthesized']).unsqueeze(0), model.config.sampling_rate).item()

    metrics[index] = {"sim": similarity, "wer": wer_value, "mos": mos, "inf/s": inferences[index]}
    print(f"{index+1}/{len(dataset)} записей обработано. Метрики новой записи: {metrics[index]}")

## Шаг 5. Визуализация данных и выводы
Визуализируем полученные данные с помощью `matplotlib` и проанализируем их. Найдем средние значения всех метрик для всех записей одновременно; затем построим графики зависимости значений метрик от определенных параметров.<br>
![title](img/metrics.png)
<br> 
<strong>SIM</strong><br>
Как можно заметить на графике средних значений метрик для всех записей, синтезированный голос почти не отличим по звучанию от голоса спикера, тенденцию чего следовало ожидать из-за природы тренировочного датасета, но, всё-таки, значение метрики SIM является удивительно высоким.<br>
<strong>WER</strong><br>
20% содержательных ошибок - неплохой результат для такого маленького датасета, но, помимо всего прочего, этот параметр имеет явную зависимость, которую мы рассмотрим позднее.<br>
<strong>MOS</strong><br>
По предсказанной системой UTMOS шкале MOS, синтез речи моделью хорошо себя показывает, выдавая среднее значение ~4 по оригинальной шкале 1-5, что отображает в общем правильную статистику - иногда в полученных аудиозаписях слышны некритичные ошибки генерации, но в целом - синтезированный голос сложно отличить от настоящего.
### Поиск зависимостей
Построим два графика, на которых попытаемся проследить зависимости значений метрик от характеристик: количества уникальных фонем в предложении и количества слов в предложении. 

In [ ]:
import matplotlib.pyplot as plt

sounds = {}
words = {}
phonetics = cmudict.dict()
for index,entry in enumerate(dataset):
    wd = len(set(preprocess_sentence(entry['transcription_normalised']).split()))
    ph = len(get_unique_phonetics(entry['transcription_normalised']))
    if ph in sounds:
        sounds[ph].append(index)
    else:
        sounds[ph] = [index]
    if wd in words:
        words[wd].append(index)
    else:
        words[wd] = [index]



bar_width = 0.2
# Позиции столбцов на оси X


def plot_statistics(parameter: dict, legend):
    data = {}
    for key in sorted(parameter):
        sim = []
        wer = []
        mos = []
        inf = []
        for index in parameter[key]:
            sim.append(int(metrics[index]["sim"]*100)) # Нормализуем долю в проценты
            wer.append(int(metrics[index]["wer"]*100))
            mos.append(int(metrics[index]["mos"]*20)) # Нормализуем шкалу 1-5 
        sim = int(sum(sim)/len(sim))
        wer = int(sum(wer)/len(wer))
        mos = int(sum(mos)/len(mos))
        data[key] = [sim, wer, mos, inf]
    r1 = range(len(data))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]
    r4 = [x + bar_width for x in r3]
    fig, ax = plt.subplots()
    bars1 = ax.bar(r1, [data[key][0] for key in data], color='b', width=bar_width, label='SIM (%)')
    bars2 = ax.bar(r2, [data[key][1] for key in data], color='r', width=bar_width, label='WER (%)')
    bars3 = ax.bar(r3, [data[key][2] for key in data], color='g', width=bar_width, label='MOS (1-5)')

    ax.set_xlabel(legend)
    ax.set_ylabel('Значение метрик')
    ax.set_title('Зависимость метрик от критерия: '+ legend)
    ax.set_xticks([r + bar_width for r in range(len(data))])
    ax.set_xticklabels(data.keys())

    ax.legend()
    plt.show()
    
plot_statistics(sounds, "Количество уникальных фонем")
plot_statistics(words, "Длина предложения (в словах)")

Рассмотрим получившиеся графики:<br>
![title](img/words.png)<br>
![title](img/phonetical.png)<br>
Ожидаемо, что чарты метрики SIM незначительно колеблются на обоих графиках, остаются в одном и том же маленьком диапазоне (90%-100%).<br>
На двух оставшихся метриках видим их явную корреляцию и зависимость от сложности предложений - как фонетической, так и лексической. Модель около безошибочно синтезирует озвучку однообразных маленьких текстов, но выдаёт все более плохие результаты по мере увеличения их сложности.
<br><br>
<i>Примечание: интересно, что по какой-то причине, значение метрики WER строится исключительно на "проглатывании" слов и фраз моделью - иногда она попросту игнорирует бОльшую часть промпта, но обрывание при этом происходит, по большей части, на моментах интонационной паузы.</i><br><br>
Подводя итоги, можно сказать, что модель Parler-TTS Mini Jenny показывает себя удивительно хорошо по общепринятым характеристикам, и в особенности при инференсе на несложных по размеру и содержании текстах. При скалировании промпта надежность аутпута модели сильно падает.